<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/202_openpyxl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

openpyxl
========

Excel ファイル
--------------

サードパティ製パッケージ [openpyxl](https://openpyxl.readthedocs.io/en/stable/) は、Excel を一切開かずに Python から直接 Excel ファイル（xlsx 形式）を操作する機能を提供する。ライセンスは MIT Licence。インストール方法は次のとおり。

``` shell
pip install openpyxl
```

Excel ファイルを構成する部品と `openpyxl` が提供するクラスは次のような対応関係にあって、以降ではインスタンスを次のように表す。

| Excel | openpyxl | インスタンス |
|:---|:---|:---|
| ブック | `Workbook` | `wb` |
| ワークシート | `Worksheet` | `ws` |
| 行 | `Row` | `row` |
| 列 | `Column` | `col` |
| セル | `Cell` | `cel` |

ワークブックの作成と保存
------------------------

Excel ファイルを新規に作成する場合、まず、`openpyxl.Workbook()` コンストラクタで `openpyxl.Workbook` オブジェクトを作成する。ワークブックを作成すると、自動でワークシートも 1 つ作成される。

``` python
from openpyxl import Workbook
wb = Workbook()
```

既存の Excel ファイルを読み込むには、`openpyxl.load_workbook()` を呼び出す。この関数は `openpyxl.Workbook` オブジェクトを返す。

``` python
openpyxl.load_workbook(filename, read_only=False, keep_vba=False, data_only=False, keep_links=True, rich_text=False)
```

| 引数 | 意味 |
|:---|:---|
| `filename` | ファイルのパス |
| `read_only` | `True` の場合、一切の書き込みができなくなる |
| `keep_vba` | `True` の場合、VBA マクロを保持する。`False`（デフォルト）の場合、保存時に VBA マクロが消えてしまう |
| `data_only` | `True` の場合、数式の計算結果のみ（値）を読み込む |
| `keep_links` | `True`（デフォルト）の場合、外部ワークブックへのリンクを保持する |
| `rich_text` | `True` に設定すると、セル内のすべてのリッチテキスト書式を保持する。デフォルトは `False` |

``` python
from openpyxl import load_workbook
wb = load_workbook('book.xlsx', keep_vba=True)
```

`openpyxl.Workbook` オブジェクトのファイルに関するメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `wb.save(filename)` | `filename` に保存する | `None` |
| `wb.close()` | ファイルが開いている場合は、ファイルを閉じる。新規作成の場合は何も起こらない | `None` |

`openpyxl.load_workbook()` 関数でファイルを読み込んだ場合は、`close()` メソッドでファイルを閉じる。 `openpyxl.Workbook` は with 文（コンテキストマネージャー）をサポートしていない。

ワークシートの操作
------------------

`openpyxl.Workbook` オブジェクトは、ワークシートに関する次の属性（読み取り専用プロパティ）を持つ。

| 属性 | 意味 |
|:---|:---|
| `wb.active` | アクティブなシート（`Worksheet` オブジェクト）を取得する |
| `wb.sheetnames` | ワークブックの中のシートの名前（文字列）のリストを取得する |
| `wb.worksheets` | ワークブックの中のシートのリスト（`Worksheet` オブジェクトのリスト）を取得する |

アクティブなシートとは、編集中のシートのことである。ファイルの保存時にアクティブであったシートは、次にファイルを開いたときにもアクティブとなる。Excel ファイルの新規作成の場合は、自動で作成されたシートがアクティブになっている。

``` python
ws = wb.active
```

`openpyxl.Workbook` オブジェクトは、イテレータープロトコルをサポートし、ワークブックの中のシートを 1 つずつ返す。`[ws for ws in wb]` は `wb.worksheets` と等価である。

In [ ]:
from openpyxl import Workbook
wb = Workbook()
assert wb.worksheets == [ws for ws in wb]

`openpyxl.Workbook` オブジェクトはマッピングオブジェクトにもなっていて、`[シート名]` を付けることで既存のシートから `Worksheet` オブジェクトを取得することができる。

``` python
ws1 = wb['Sheet1']
```

新規ファイル作成時はデフォルトでシート名 `'Sheet'` のワークシートが存在しているので、不要な場合は以下で削除する。

``` python
del wb["Sheet"]
```

`openpyxl.Workbook` オブジェクトは、ワークシートの操作に関する次のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `wb.create_sheet(title=None, index=None)` | ワークシートを作成する。シート名は自動で付けられるが、`title` で指定できる。`index` にシートを<br />挿入する位置を整数で指定でき、0 を指定すると左端に作成する。デフォルトでは右端に作成する | `Worksheet` |
| `wb.remove(worksheet)` | ワークシートを削除する。`del wb[sheetname]` と等価 | `None` |
| `wb.index(worksheet)` | ワークシートの位置を返す | `int` |
| `wb.move_sheet(sheet, offset=0)` | `sheet` の位置を `offset` だけ移動する | `None` |
| `wb.copy_worksheet(from_worksheet)` | 引数に指定したワークシートをコピーする。作成したワークシートを返す | `Worksheet` |

`Worksheet` オブジェクトの属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `ws.title` | シート名を表す文字列。この属性は変更可能で、代入によりシート名を変更できる |
| `ws.max_column` | ワークシートの使用領域での最大の列番号である整数 |
| `ws.min_column` | ワークシートの使用領域での最小の列番号である整数 |
| `ws.max_row` | ワークシートの使用領域での最大の行番号である整数 |
| `ws.min_row` | ワークシートの使用領域での最小の行番号である整数 |

ここで使用領域とは、値が書き込まれたセルをすべて囲む最小の四角形領域のことである。**列番号も行番号も 1 からはじまる（0 ではないので注意）**。

行と列
------

### 行と列の操作 ###

`Worksheet` オブジェクトが作成されると、`Row` オブジェクトと `Column` オブジェクトも自動的に作成される。

`Worksheet` オブジェクトに対して、`[1]` のような Excel の行番号を使ってインデックスを付けると、行（`Row` オブジェクト）を取得する。行番号は 1 から始まる（0 ではないので注意）。

``` python
row = ws[1]
```

`Worksheet` オブジェクトに対して、`['A']` のような Excel の列指定文字列を使ってキーを付けると、列（`Column` オブジェクト）を取得する。

``` python
col = ws['A']
```

以下の `Worksheet` オブジェクトの属性を使って、ワークシートの行および列の表示設定ができる:

| 属性 | 意味 |
|:---|:---|
| `ws.row_dimensions` | `RowDimensions` オブジェクトのコンテナを参照する。`ws.row_dimensions[1]` で行番号 1 の行の `RowDimensions` オブジェクトにアクセス<br />できる |
| `ws.column_dimensions` | `ColumnDimensions` オブジェクトのコンテナを参照する。`ws.column_dimensions['B']` で `'B'` 列の `ColumnDimensions` オブジェクトにア<br />クセスできる |
| `ws.freeze_panes` | Excel のセル指定文字列（例 `'B2'`）を指定すると、そのセルを左上位置とする四角形領域がスクロール可能となり、その他の行と列（Γ型<br />の領域）の位置が固定される |

`RowDimensions` オブジェクトは行の表示に関する情報を保持し、その `height` 属性で行の高さ（ポイント単位）を指定できる。また、`ColumnDimensions` オブジェクトは列の表示に関する情報を保持し、その `width` 属性で列の幅（文字数単位）を指定できる。

``` python
ws.row_dimensions[1].height = 60  # 行の高さを 60 ポイント
ws.column_dimensions['B'].width = 30  # 列の幅を 30 文字
```

`ws.freeze_panes` が `'A1'` 以外の場合、スクロールされない行または列が存在することになる。固定を解除したい場合は、`'A1'` を指定すればよい。

``` python
ws.freeze_panes = 'A2'  # 1 行目だけを固定する
ws.freeze_panes = 'B1'  # A 列だけを固定する
ws.freeze_panes = 'B2'  # 1 行目と A 列を固定する
ws.freeze_panes = 'A1'  # 固定を解除する
```

`Worksheet` オブジェクトは、行および列を操作する次のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `ws.insert_cols(idx, amount=1)` | 列番号 `idx` の位置に列を挿入する。挿入する列の数を `amount` で指定する（デフォルトは 1） | `None` |
| `ws.insert_rows(idx, amount=1)` | 行番号 `idx` の位置に行を挿入する。挿入する行の数を `amount` で指定する（デフォルトは 1） | `None` |
| `ws.delete_cols(idx, amount=1)` | 列番号 `idx` の列を削除する。`amount` に削除する列の数を指定できる | `None` |
| `ws.delete_rows(idx, amount=1)` | 列番号 `idx` の行を削除する。`amount` に削除する行の数を指定できる | `None` |

### 列指定文字列と列番号に関する関数 ###

`column_index_from_string(str_col)` 関数は、Excel の列指定文字列を列番号に変換する。

In [ ]:
from openpyxl.utils import column_index_from_string
column_index_from_string('A')

1

`get_column_letter(idx)` 関数は、列番号を Excel の列指定文字列に変換する。

In [ ]:
from openpyxl.utils import get_column_letter
get_column_letter(3)

'C'

セルの操作
----------

### セルの選択 ###

`Worksheet` オブジェクトに対して、`['A1']` のような Excel のセル指定文字列を使ってキーを付けると、セル（`Cell` オブジェクト）を取得することができる。

``` python
cel = ws['A1']  # cel には Cell オブジェクトが格納される
```

代入文の左辺では特別にセルの値にアクセスすることになり、セルの値が書き換えられる。

``` python
ws['A1'] = 4  # セルの内容が書き換えられる
```

セルの値は、設定されてないとき、つまり未使用の状態では `None` となっている。

`Worksheet` オブジェクトに与えるキーを、Excel の範囲指定文字列（シートの四角形領域の左上のセルと右下のセルのセル指定文字を `:` で連結したもの）とすると、その領域にある行のタプルを取得する。各行は列のセルからなるタプルである。

``` python
cell_range = ws['A1:C2']
# 以下と同じ
# ((ws['A1'], ws['B1'], ws['C1']),
#  (ws['A2'], ws['B2'], ws['C2']))
```

`ws[row_index]` でアクセスされる `Row` オブジェクトはセルのシーケンスであり、タプルと同様にインデックスでセルを参照できる（インデックス番号は 0 から始まることに注意）。

In [ ]:
from openpyxl import Workbook
ws = Workbook().active
assert ws[1][0] is ws['A1']

`Worksheet` オブジェクトの `rows` 属性は、ワークシートの使用領域での各行（セルからなるタプル）を 1 行ずつ返すジェネレーターを参照する。

``` python
for row in ws.rows:
	for cel in row:
        pass
```

`Worksheet` オブジェクトの `columns` 属性は、ワークシートの使用領域での各列（セルからなるタプル）を 1 列ずつ返すジェネレーターを参照する。

``` python
for col in ws.columns:
    for cel in col:
        pass
```

また、`Worksheet` オブジェクトは、セル選択に関する次のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `ws.cell(row, column, value=None)` | 行番号 `row`、列番号 `column` の `Cell` オブジェクトを取得する。`value` オプションを指定<br />すると、その値でセルの内容を書き換える | `Cell` |
| `ws.iter_rows(min_row=None, max_row=None,`<br />` min_col=None, max_col=None, values_only=False)` | 四角形領域のセルを含む行を 1 行ずつ yield するジェネレーターを返す。`values_only` が<br /> `True` の場合、 yield された行から得られるのは `Cell` オブジェクトではなくセルの値となる | ｼﾞｪﾈﾚｰﾀｰ |
| `ws.iter_cols(min_col=None, max_col=None,`<br />` min_row=None, max_row=None, values_only=False)` |四角形領域のセルを含む列を 1 列ずつ yield するジェネレーターを返す。`values_only` が<br /> `True` の場合、 yield された列から得られるのは `Cell` オブジェクトではなくセルの値となる | ｼﾞｪﾈﾚｰﾀｰ |

四角形領域を指定して行ごとの操作をする例:

In [ ]:
from openpyxl import Workbook
ws = Workbook().active
for row in ws.iter_rows(min_row=1, max_col=3, max_row=2):
    for cel in row:
        print(cel)

<Cell 'Sheet'.A1>
<Cell 'Sheet'.B1>
<Cell 'Sheet'.C1>
<Cell 'Sheet'.A2>
<Cell 'Sheet'.B2>
<Cell 'Sheet'.C2>


`ws.rows` 属性や `ws.columns` 属性を使う場合、ワークシートの使用領域内にあるセルしかアクセスできない。新規作成の場合でループ操作をしたい場合は、`ws.iter_rows()`、`ws.iter_cols()` メソッドで範囲指定する必要がある。

### セルの基本操作 ###

以下の `Cell` オブジェクトの属性を使って、セルに関する情報を取得できる:

| 属性 | 意味 |
|:---|:---|
| `cel.value` | セルの値 |
| `cel.row` | セルの行数（縦軸の座標） |
| `cel.column` | セルの列数（横軸の座標） |
| `cel.column_letter` | Excel の列指定文字列（例 `'A'`） |
| `cel.coordinate` |  Excel のセル指定文字列（例 `'A5'`） |

既存の Excel ファイルで、セルに対し `cel.value` から取得できる値の型は、データに応じて `int`, `float`, `str` などとなっている。とくに、セルの内容が日付（と時間）の場合、`cel.value` から `datetime.datetime` オブジェクトが得られ、時間の場合は `datetime.time` オブジェクトが得られる。値が設定されていない場合、`cel.value` は `None` になっている。

`cel.value` 属性にセルの値を書き込むことができる。また、Excel の数式を文字列として書き込むこともできる。

``` python
numbers = [300, 500, 200]
row_base = 1
column = 1
for i, number in enumerate(numbers):
    ws.cell(row_base + i, column).value = number  # A1, A2, A3 のセルに書き込む
i += 1
ws.cell(row_base + i, column).value = "=SUM(A1:A3)"  # sum(numbers) と計算結果を書き込んでもよい
```

`Worksheet` オブジェクトの `append(iterable)` メソッドを使うと、リストや辞書から一気に最終行に追加する形で書き込むことができる。`iterable` がリストの場合、`'A'` 列から連続するセルに書き込む。`iterable` が辞書の場合、キー（列指定文字列または列番号）の列にあるセルに書き込む。

``` python
ws.append(['This is A1', 'This is B1', 'This is C1'])
ws.append({'A' : 'This is A1', 'C' : 'This is C1'})
ws.append({1 : 'This is A1', 3 : 'This is C1'})
```

### セルの結合 ###

`Worksheet` オブジェクトは、セルの結合に関する次のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `ws.merge_cells(range_string=None, start_row=None, `<br />`start_column=None, end_row=None, end_column=None)` | 範囲内のセルを結合する。`range_string` に Excel の範囲指定文字列を指定するか、あるい<br />は `start_row`, `start_column`, `end_row`, `end_column` により左上の行と列、右下の行と列<br />を指定する | `None` |
| `ws.unmerge_cells(range_string=None, start_row=None,`<br />` start_column=None, end_row=None, end_column=None)` | 範囲内のセル結合を解除する。引数の役割は `ws.merge_cells()` と同じ | `None` |

結合後のセルは、範囲内の左上のセルが代表することになる。範囲内のその他のセルに書き込まれた値は、結合によって失われることに注意する。

In [ ]:
from openpyxl import Workbook
ws = Workbook().active
ws.append(["This is A1", "This is B1", "This is C1"])
assert ws.cell(1, 2).value == "This is B1"
ws.merge_cells("A1:C1")
assert ws.cell(1, 2).value is None  # 結合後は消去される
assert ws.cell(1, 1).value == "This is A1"  # 結合後は左上のセルが代表する

セル結合を解除するときは、結合された範囲を正確に指定しないとエラーが発生する。

セルの書式設定
--------------

以下の `Cell` オブジェクトの属性を使って、セルの書式設定ができる:

| 属性 | 意味 |
|:---|:---|
| `cel.font` | セルのフォント |
| `cel.fill` | セルの背景色 |
| `cel.number_format` | セルの表示形式 |
| `cel.alignment` | セルの文字の位置 |
| `cel.border` | セルの枠線 |
| `cell.hyperlink` | ハイパーリンク |

### セルのフォント ###

`Cell` オブジェクトの `font` 属性に `openpyxl.styles.Font` オブジェクトを指定することでセルのフォントを設定できる。

``` python
openpyxl.styles.Font(name=None, sz=None, b=None, i=None, charset=None, u=None, strike=None, color=None, scheme=None, family=None, size=None, bold=None, italic=None, strikethrough=None, underline=None, vertAlign=None, outline=None, shadow=None, condense=None, extend=None)
```

| 引数 | 意味 |
|:---|:---|
| `name` | フォント名を文字列で指定する。この引数は位置引数で指定してもよい |
| `size` または `sz` | 文字サイズを整数で指定する |
| `bold` または `b`| `True` の場合、太文字になる |
| `italic` または `i` | `True` の場合、斜体になる |
| `color` | 文字色をカラーコード（`rrggbb` 形式）で指定する |
| `strike` または `strikethrough` | `True` の場合、打ち消し線を追加する |
| `underline` または `u` | 下線を追加する。`'single'` なら一重線、`'double'` なら二重線となる |

``` python
from openpyxl.styles import Font
cel.font = Font(
    name="メイリオ",
    size=20,
    bold=False,
    italic=False,
    underline="single",
    strike=False,
    color="FF0000",
)
```

以下のコードは、CSV 形式のファイルからデータを読み込み、Excel 形式のファイルに変換している。 1 行目（ヘッダー行）を固定し、その行のフォントを太字に設定している。スクリプトを実行すると、Colab のローカルファイルに `test_font.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [1]:
import csv
from openpyxl import Workbook
from openpyxl.styles import Font

# ワークブックの新規作成
wb = Workbook()
ws = wb.active
row_base = 1
column_base = 1

# 表示設定
ws.freeze_panes = "A2"
bold_font = Font(bold=True)

# CSV からデータを入力
with open("sample_data/california_housing_test.csv", "r", encoding="utf-8", newline="") as f:
    reader = csv.reader(f)
    for i, row_data in enumerate(reader):
        for j, val in enumerate(row_data):
            if i == 0:
                cel = ws.cell(row_base + i, column_base + j)
                cel.value = val
                cel.font = bold_font
                ws.column_dimensions[cel.column_letter].width = 20
            else:
                ws.cell(row_base + i, column_base + j).value = float(val)  # 数値化

wb.save("test_font.xlsx")

from google.colab import files
files.download("/content/test_font.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### セルの背景色 ###

`Cell` オブジェクトの  `fill` 属性に `openpyxl.styles.PatternFill` オブジェクトを指定することでセルの背景色を設定できる。

``` python
openpyxl.styles.PatternFill(patternType=None, fgColor=Color(), bgColor=Color(), fill_type=None, start_color=None, end_color=None)
```

| 引数 | 意味 |
|:---|:---|
| `patternType` または `fill_type` | 以下を指定できる<br /><br />・`'solid'`: 単色<br /><br />・`'darkDown'`: 右下がりの斜線（＼）　太線　縞<br /><br />・`'darkGray'`: 75 %灰色<br /><br />・`'darkGrid'`: 斜めの格子<br /><br />・`'darkHorizontal'`: 横縞　太線<br /><br />・`'darkTrellis'`: 斜めの格子 太線<br /><br />・`'darkUp'`: 左下がりの斜線（／）　太線　縞<br /><br />・`'darkVertical'`: 縦縞　太線<br /><br />・`'gray0625'`: 6.25 %灰色<br /><br />・`'gray125'`: 12.5 %灰色<br /><br />・`'lightDown'`: 右下がりの斜線（＼）　細線　縞<br /><br />・`'lightGray'`: 25 %灰色<br /><br />・`'lightGrid'`: 格子<br /><br />・`'lightHorizontal'`: 横縞　細線<br /><br />・`'lightTrellis'`: 斜めの格子 細線<br /><br />・`'lightVertical'`: 縦縞　細線<br /><br />・`'lightUp'`: 左下がりの斜線（／）　細線　縞<br /><br />・`'mediumGray'`: 50 %灰色 |
| `fgColor` または `start_color` | 背景色に使う色をカラーコード（`rrggbb` 形式）で指定する |
| `bgColor` または `end_color` | 背景色に使う色をカラーコード（`rrggbb` 形式）で指定する |

単色で塗りつぶす場合は、`patternType`（または `fill_type`）に `'solid'` を指定し、`fgColor`（または `start_color`）に塗りつぶす色を指定する（`bgColor` は使われない）。

``` python
from openpyxl.styles import PatternFill
cel.fill = PatternFill("solid", "33FFFF")
```

### セルの表示形式 ###

`Cell` オブジェクトの `number_format` 属性に次のような文字列を指定することでセルの表示形式を設定できる。

| 文字列定数 | 意味 |
|:---|:---|
| `'General'` | 標準。数値や文字列を書き込んだ時のデフォルト |
| `'yyyy-mm-dd h:mm:ss'` | 日付と時間（1970-01-01 15:00:00 形式）。`datetime.datetime` オブジェクトを書き込んだ時のデフォルト |
| `'yyyy-mm-dd'` | 日付（1970-01-01 形式）。`datetime.date` オブジェクトを書き込んだ時のデフォルト |
| `'h:mm:ss'` | 時間（15:00:00 形式）。`datetime.time` オブジェクトを書き込んだ時のデフォルト |
| `'0'` | 数値（小数点以下の桁数 0） |
| `'0.00'` | 数値（小数点以下の桁数 2） |
| `'#,##0'` | 3 桁区切り記号付き数値（小数点以下の桁数 0） |
| `'#,##0.00''` | 3 桁区切り記号付き数値（小数点以下の桁数 2） |
| `'0%'` | パーセンテージ（小数点以下の桁数 0） |
| `'0.00%'` | パーセンテージ（小数点以下の桁数 2） |
| `'0.00E+00'` | 指数（小数点以下の桁数 2） |
| `'yyyy/m/d'` | 日付（1970/1/1 形式） |
| `'yyyy"年"m"月"d"日"'` | 日付（1970年1月1日 形式） |
| `'h:mm AM/PM',` | 時間（3:00 PM 形式) |
| `'h:mm:ss AM/PM'` | 時間（3:00:00 PM 形式） |
| `'h:mm'` | 時間（15:00 形式） |
| `'@'` | 文字列 |

セルに日付や時間を書き込むときは、`datetime.datetime` オブジェクトや、`datetime.date` オブジェクト、`datetime.time` オブジェクトを使わなければならない。

``` python
import datetime
cel.value = datetime.datetime.now()
cel.number_format = 'yyyy"年"m"月"d"日"'
```

### セルの文字の位置 ###

`Cell` オブジェクトの `alignment` 属性に `openpyxl.styles.Alignment` オブジェクトを指定することでセルの文字の位置を設定できる。

``` python
openpyxl.styles.Alignment(horizontal=None, vertical=None, textRotation=0, wrapText=None, shrinkToFit=None, indent=0, relativeIndent=0, justifyLastLine=None, readingOrder=0, text_rotation=None, wrap_text=None, shrink_to_fit=None, mergeCell=None)
```

| 引数 | 意味 |
|:---|:---|
| `horizontal`| 横方向の位置。次の値を指定できる<br />・`'general'`: 標準<br />・`'left'`: 左揃え<br />・`'center'`: 中央揃え<br />・`'right'`: 右揃え<br />・`'justify'`: 両端揃え<br />・`'fill'`: 繰り返し<br />・`'distributed'`: 均等割り付け<br />・`'centerContinuous'`: 選択範囲内で中央 |
| `vertical` | 縦方向の位置。次の値を指定できる<br />・`'top'`: 上揃え<br />・`'center'`: 中央揃え<br />・`'bottom'`: 下揃え<br />・`'justify'`: 両端揃え<br />・`'distributed'`: 均等割り付け |
| `textRotation` または `text_rotation` | 反時計回りで回転する角度。 0 から 180 までの範囲内の整数を指定できる |
| `wrapText` または `wrap_text` | `True` の場合、セル幅を超えないように改行する。`False` の場合、改行されない |

``` python
from openpyxl.styles import Alignment
cel.alignment = Alignment(horizontal="center", vertical="center")
```

### セルの枠線 ###

`Cell` オブジェクトの `border` 属性に `openpyxl.styles.Border` オブジェクトを指定することでセルの枠線を設定できる。

``` python
openpyxl.styles.Border(left=None, right=None, top=None, bottom=None, diagonal=None, diagonal_direction=None, vertical=None, horizontal=None, diagonalUp=False, diagonalDown=False, outline=True, start=None, end=None)
```

| 引数 | 意味 |
|:---|:---|
| `left` | 左側の枠線 |
| `right` | 右側の枠線 |
| `top` | 上側の枠線 |
| `bottom` | 下側の枠線 |
| `diagonal` | 対角線の枠線 |
| `diagonal_direction` | `diagonal` を指定する場合に対角線の方向を指定する<br /><br />・`0`: なし（デフォルト）<br /><br />・`1`: 左上→右下 <br /><br />・`2`: 右上→左下: <br /><br />・`3`: 両方 |
| `diagonalUp` | `diagonal` を指定する場合に `diagonalUp=True` なら右上がり（／）の斜線を引く |
| `diagonalDown` | `diagonal` を指定する場合に `diagonalDown=True` なら右下がり（＼）の斜線を引く |
| `outline` | `True` の場合、セルの外周に枠線を引く |
| `vertical` | 縦の内部枠線。主に結合セルや範囲に対して使用 |
| `horizontal` | 横の内部枠線。主に結合セルや範囲に対して使用 |


`left`、`right`、`top`、`bottom`、`diagonal` 各引数に指定する枠線は、`openpyxl.styles.Side` オブジェクトを指定する。

``` python
openpyxl.styles.borders.Side(style=None, color=None, border_style=None)
```

| 引数 | 意味 |
|:---|:---|
| `style` または `border_style` | 枠線の形式を指定する。次の値を指定できる<br />・`'none'`: 枠線なし（デフォルト）<br />・`'dotted'`: 点線<br />・`'dashed'`: 破線<br />・`'dashDot'`: 一点鎖線<br />・`'dashDotDot'`: 二点鎖線<br />・`'hair'`: 極細線<br />・`'thin'`: 細線<br />・`'medium'`: 中細線<br />・`'mediumDashed'`: 中細破線<br />・`'mediumDashDot'`: 中一点鎖線<br />・`'mediumDashDotDot'`: 中二点鎖線<br />・`'slantDashDot'`: 一点鎖斜線<br />・`'thick'`: 太線<br />・`'double'`: 二重線 |
| `color` | 枠線の色をカラーコード（`rrggbb` 形式）で指定する |

``` python
from openpyxl.styles import Side, Border
side_none = Side("none")
side_thick = Side("thick", "FF0000")
cel.border = Border(left=side_thick, right=side_none, top=side_thick, bottom=side_thick)
```

### セルのハイパーリンク ###

`Cell` オブジェクトの `hyperlink` 属性でハイパーリンクを設定できる。 `cell.value` 属性に直接 Excel  の `HYPERLINK` 関数を指定することでもハイパーリンクを設定できる。

``` python
# A1 に Sheet2 という文字列を設定して、リンク先に同ドキュメント内の Sheet2 シートの A1 セルを指定
ws["A1"].value = "Sheet2"
ws["A1"].hyperlink = "test.xlsx#sheet2!A1"

# A2に Google という文字列を設定して、リンク先に Google のアドレスを指定
ws['A2'].value = "Google"
ws['A2'].hyperlink = "https://www.google.com"

# HYPERLINK 関数自体を設定することも可能
ws['A3'].value = '=HYPERLINK("https://www.amazon.co.jp/", "Amazon")'
```

条件付き書式
------------

Excel の**条件付き書式**とは、セルの値や数式の結果に応じて自動的に書式（フォント、背景色、枠線など）を変える機能である。データの傾向や異常値を視覚的にすぐ判断できるようにするためによく使われる。

条件付き書式を設定するには、 `Worksheet` オブジェクトの `conditional_formatting` プロパティから参照される `ConditionalFormattingList` オブジェクトを設定する。 `ConditionalFormattingList` オブジェクトは「ワークシートの範囲」と「条件付き書式のリスト」のペアを保持するイテラブルなオブジェクトである。「条件付き書式のリスト」に追加した順番は条件付き書式の優先順位になる。

「条件付き書式のリスト」に条件付き書式を追加するには、 `add()` メソッドを使う。

``` python
ws.conditional_formatting.add(range_string, cfRule)
```

| 引数 | 意味 |
|:---|:---|
| `range_string` | Excel の範囲指定文字列 |
| `cfRule` | `openpyxl.formatting.rule.Rule` オブジェクト |

`Rule` クラスは条件付き書式を表す。コンストラクタは次のとおり。

``` python
openpyxl.formatting.rule.Rule(type, dxfId=None, priority=0, stopIfTrue=None, aboveAverage=None, percent=None, bottom=None, operator=None, text=None, timePeriod=None, rank=None, stdDev=None, equalAverage=None, formula=(), colorScale=None, dataBar=None, iconSet=None, extLst=None, dxf=None)
```

主な引数は以下の通り。

| 引数 | 意味 |
|:---|:---|
| `type` | `type` 属性（条件のタイプ）の値を文字列定数として与える |
| `stopIfTrue` | `stopIfTrue` 属性の値をブール値として与える。`True` の場合、この条件が真のときに「条件付き書式のリスト」内で後ろの条件付き書式は適用され<br />ない |

第 1 引数 `type` は必ず指定する必要があるが、それ以外の引数は `stopIfTrue` 引数を除いて `type` の値に応じて必要があれば指定することになる。 `stopIfTrue` 引数は条件付き書式の適用に関するオプションである。

条件付き書式の効果には「セルの強調表示」「データバー」「カラースケール」「アイコンセット」の 4 種類があり、 `type` 引数で種類を設定することになるが、「セルの強調表示」以外のものは `openpyxl` で Excel の設定を網羅していなかったり、不具合があったり、そもそも Excel での利用が少ないため、あまり使われていない。

「セルの強調表示」は、`openpyxl.styles.differential.DifferentialStyle` クラスを使って設定する。コンストラクタは次のとおり。

``` python
openpyxl.styles.differential.DifferentialStyle(font=None, numFmt=None, fill=None, alignment=None, border=None, protection=None, extLst=None)
```

主な引数は以下の通り。

| 引数 | 意味 |
|:---|:---|
| `font` | 条件に一致したセルに適用するフォントスタイル（`Font` オブジェクト） |
| `fill` | 条件に一致したセルに適用する塗りつぶし（`PatternFill` など） |
| `border` | 条件に一致したセルに適用する枠線（`Border` オブジェクト） |

条件のタイプ（`Rule` の `type` 属性）が「セルの強調表示」を行うものである場合、 `Rule` クラスのコンストラクタの `dxf` 引数に `DifferentialStyle` オブジェクトを渡すか、あるいは `Rule` オブジェクト生成後にインスタンスの `dxf` 属性に `DifferentialStyle` オブジェクトを与えてもよい。

### セルの値による強調表示 ###

セルの値が比較条件を満たす場合に強調表示の書式に変えるには、 `Rule` クラスのコンストラクタを以下のように呼び出す:

  1. `type` 引数に `'cellIs'` を指定し
  2. `operator` 引数に以下の文字列定数のいずれかを指定し
  3. `formula` 引数に条件に使う値や Excel 数式をシーケンスで指定し
  4. `dxf` 引数に `DifferentialStyle` オブジェクトを指定する（省略してオブジェクト生成後に `dxf` 属性に `DifferentialStyle` オブジェクトを与えてもよい）

| `operator` の値 | 意味 |
|:---|:---|
| `'greaterThan'` | `formula[0]` の値より大きい |
| `'greaterThanOrEqual'` | `formula[0]` の値以上 |
| `'lessThan'` | `formula[0]` の値より小さい |
| `'lessThanOrEqual'` | `formula[0]` の値以下 |
| `'equal'` | `formula[0]` の値に等しい |
| `'notEqual'` | `formula[0]` の値に等しくない |
| `'between'` | `formula[0]` から `formula[1]` までの範囲（境界を含む） |
| `'notBetween'` | `formula[0]` から `formula[1]` までの範囲外（境界を含まない） |

`'between'` と `'notBetween'` の場合、`formula` は 2 要素のシーケンスになる。

以下のコードは、値が 5 より大きいセルの書式を濃い赤の文字、明るい赤の背景に変更している。スクリプトを実行すると、Colab のローカルファイルに `cell_is_rule.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

wb = Workbook()
ws = wb.active

# 値の入力
for i in range(1, 11):
    ws[f"A{i}"] = i

# 5より大きいセルに濃い赤の文字、明るい赤の背景
rule = Rule(type='cellIs', operator='greaterThan', formula=['5'])
font = Font(color="9C0006")
fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
rule.dxf = DifferentialStyle(font=font, fill=fill)

# 範囲にルールを適用（A1～A10）
ws.conditional_formatting.add('A1:A10', rule)

wb.save("cell_is_rule.xlsx")

from google.colab import files
files.download("/content/cell_is_rule.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

セルの値によるセルの強調表示ルールはよく使うものなので、次の便利関数が提供されている。

``` python
openpyxl.formatting.rule.CellIsRule(operator=None, formula=None, stopIfTrue=None, font=None, border=None, fill=None)
```

この関数は、内部で `DifferentialStyle` オブジェクトを作成し、 `'cellIs'` タイプの `Rule` オブジェクトを生成して、その `dxf` 属性に `DifferentialStyle` オブジェクトを与えた上で返す。 `font` 引数、`border` 引数、`fill` 引数は `DifferentialStyle` クラスのコンストラクタに渡される。

上記のコードのうち、インポート部分と条件付き書式設定部分

``` python
from openpyxl.styles import Font, PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

rule = Rule(type='cellIs', operator='greaterThan', formula=['5'])
font = Font(color="9C0006")
fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
rule.dxf = DifferentialStyle(font=font, fill=fill)
```

これは `CellIsRule()` 関数を使って次のように簡単に書ける。

``` python
from openpyxl.styles import Font, PatternFill
from openpyxl.formatting.rule import CellIsRule

font = Font(color="9C0006")
fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
rule = CellIsRule(operator='greaterThan', formula=['5'], font=font, fill=fill)
```

### 文字列による強調表示 ###

文字列が特定条件を満たす場合に強調表示の書式に変えるには、 `Rule` クラスのコンストラクタを以下のように呼び出す:

  1. `type` 引数に以下の文字列定数のいずれかを指定し
  2. `text` 引数に条件に使う文字列を指定し
  3. `dxf` 引数に `DifferentialStyle` オブジェクトを指定する（省略してオブジェクト生成後に `dxf` 属性に `DifferentialStyle` オブジェクトを与えてもよい）

| `type` の値 | 意味 |
|:---|:---|
| `'containsText'` | `text` の値を含む |
| `'notContainsText'` | `text` の値を含まない |
| `'beginsWith'` | `text` の値で始まる |
| `'endsWith'` | `text` の値が末尾 |

以下のコードは、`'重要'` を含む場合に黄色の背景に変更している。スクリプトを実行すると、Colab のローカルファイルに `test_contains_text.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

wb = Workbook()
ws = wb.active

# デモ用のデータ
ws["A1"] = "これは重要なメモです"
ws["A2"] = "普通のメモ"
ws["A3"] = "非常に重要"

# 条件付き書式のルール作成（"重要" を含む場合）
rule = Rule(type="containsText", text="重要")
fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
rule.dxf = DifferentialStyle(fill=fill)

ws.conditional_formatting.add("A1:A10", rule)
wb.save("test_contains_text.xlsx")

from google.colab import files
files.download("/content/test_contains_text.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 上位/下位による強調表示 ###

上位/下位ルールを満たす場合に強調表示の書式に変えるには、 `Rule` クラスのコンストラクタを以下のように呼び出す:

  1. `type` 引数に文字列定数 `'top10'` を指定し
  2. `rank` 引数に条件で使用する整数を指定し（指定を省略した場合のデフォルト値は `10`）
  3. `percent` 引数に条件で使用するブール値を指定し（指定を省略した場合のデフォルト値は `False`）
  4. `bottom` 引数に条件で使用するブール値を指定し（指定を省略した場合のデフォルト値は `False`）
  5. `dxf` 引数に `DifferentialStyle` オブジェクトを指定する（省略してオブジェクト生成後に `dxf` 属性に `DifferentialStyle` オブジェクトを与えてもよい）

上位/下位ルールと各引数の対応関係は以下の通り。

| 上位/下位ルール | `type` | `rank` | `percent` | `bottom` |
|:---|:---|:---|:---|:---|
| 上位 10 項目 | `'top10'` | `10`（デフォルト） | `False`（デフォルト） | `False`（デフォルト） |
| 上位 10 % | `'top10'` | `10`（デフォルト） | `True` | `False`（デフォルト） |
| 下位 10 項目 | `'top10'` | `10`（デフォルト） | `False`（デフォルト） | `True` |
| 下位 10 % | `'top10'` | `10`（デフォルト） | `True` | `True` |

以下のコードは、下位 3 項目を赤の文字に変更している。スクリプトを実行すると、Colab のローカルファイルに `test_bottom.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [4]:
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

wb = Workbook()
ws = wb.active

# 値の入力
for i in range(1, 11):
    ws[f"A{i}"] = i

# 下位 3 項目に赤の文字
rule = Rule(type='top10', rank=3, bottom=True, stopIfTrue=True)
font = Font(color="FF0000")
rule.dxf = DifferentialStyle(font=font)

# 範囲にルールを適用（A1～A10）
ws.conditional_formatting.add('A1:A10', rule)

wb.save("test_bottom.xlsx")

from google.colab import files
files.download("/content/test_bottom.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 平均より上/下による強調表示 ###

平均より上/下ルールを満たす場合に強調表示の書式に変えるには、 `Rule` クラスのコンストラクタを以下のように呼び出す:

  1. `type` 引数に文字列定数 `'aboveAverage'` を指定し
  2. `aboveAverage` 引数に条件で使用するブール値を指定し（指定を省略した場合のデフォルト値は `True`）
  3. `equalAverage` 引数に条件で使用するブール値を指定し（指定を省略した場合のデフォルト値は `False`）
  4. `dxf` 引数に `DifferentialStyle` オブジェクトを指定する（省略してオブジェクト生成後に `dxf` 属性に `DifferentialStyle` オブジェクトを与えてもよい）

平均より上/下ルールと各引数の対応関係は以下の通り。

| 上位/下位ルール | `type` | `aboveAverage` | `equalAverage` |
|:---|:---|:---|:---|
| 平均を上回る | `'aboveAverage'` | `True`（デフォルト） | `False`（デフォルト） |
| 平均以上 | `'aboveAverage'` | `True`（デフォルト） | `True` |
| 平均を下回る | `'aboveAverage'` | `False` | `False`（デフォルト） |
| 平均以下 | `'aboveAverage'` | `False` | `True` |

以下のコードは、平均を下回る場合に水色の背景に変更している。スクリプトを実行すると、Colab のローカルファイルに `test_below_average.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [5]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

wb = Workbook()
ws = wb.active

# 値の入力
for i in range(1, 12):
    ws[f"A{i}"] = i

# 平均を下回る場合に水色の背景
rule = Rule(type='aboveAverage', aboveAverage=False)
fill = PatternFill(start_color='C0E4F5', end_color='C0E4F5', fill_type='solid')
rule.dxf = DifferentialStyle(fill=fill)

# 範囲にルールを適用（A1～A11）
ws.conditional_formatting.add('A1:A11', rule)

wb.save("test_below_average.xlsx")

from google.colab import files
files.download("/content/test_below_average.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 重複する値/一意の値による強調表示 ###

セルの値が重複する場合（または重複しない場合）に強調表示の書式に変えるには、 `Rule` クラスのコンストラクタを以下のように呼び出す:

  1. `type` 引数に `'duplicateValues'`（または `'uniqueValues'`）を指定し
  2. `dxf` 引数に `DifferentialStyle` オブジェクトを指定する（省略してオブジェクト生成後に `dxf` 属性に `DifferentialStyle` オブジェクトを与えてもよい）

### 数式による強調表示 ###

Excel 数式を使って強調表示のための新しい条件（ルール）を作ることもできる。 `Rule` クラスのコンストラクタを以下のように呼び出す:

  1. `type` 引数に `'expression'` を指定し
  2. `formula` 引数に Excel 数式 をシーケンスで指定し
  3. `dxf` 引数に `DifferentialStyle` オブジェクトを指定する（省略してオブジェクト生成後に `dxf` 属性に `DifferentialStyle` オブジェクトを与えてもよい）

新しいルールのためにも、次の便利関数が提供されている。

``` python
openpyxl.formatting.rule.FormulaRule(formula=None, stopIfTrue=None, font=None, border=None, fill=None)
```

この関数は、内部で `DifferentialStyle` オブジェクトを作成し、 `'expression'` タイプの `Rule` オブジェクトを生成して、その `dxf` 属性に `DifferentialStyle` オブジェクトを与えた上で返す。 `font` 引数、`border` 引数、`fill` 引数は `DifferentialStyle` クラスのコンストラクタに渡される。

以下のコードは、奇数行を水色の背景に変更している。スクリプトを実行すると、Colab のローカルファイルに `test_striped.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [8]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule

wb = Workbook()
ws = wb.active

# 値の入力
for i in range(1, 11):
    ws[f"A{i}"] = i

# 奇数行を水色の背景
fill = PatternFill(start_color='C0E4F5', end_color='C0E4F5', fill_type='solid')
rule = FormulaRule(['MOD(ROW(),2)<>0'], fill=fill)

# 範囲にルールを適用（A1～A10）
ws.conditional_formatting.add('A1:A10', rule)

wb.save("test_striped.xlsx")

from google.colab import files
files.download("/content/test_striped.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

画像の挿入
----------

ワークシートに画像を挿入するには、 `Worksheet` オブジェクトの `add_image()` メソッドを使用する。

``` python
ws.add_image(img, anchor=None)
```

| 引数 | 意味 |
|:---|:---|
| `img` | 挿入する画像を `openpyxl.drawing.image.Image` オブジェクトで指定する |
| `anchor` | 挿入位置をセル指定文字列で指定する |

`openpyxl.drawing.image.Image` のコンストラクタは、唯一の引数として画像ファイルのパスを受け付ける。その引数はローカルのファイルパスでなければならず、 Web 上の URL を指定することはできない。 対応する画像形式は Pillow と同じ。

`ws.add_image()` メソッドで画像を挿入すると画像自体が Excel ファイルに取り込まれるので、 `wb.save()` メソッドで保存した後ならソースの画像ファイルを移動したり削除しても Excel ファイルの中の画像に影響はない。

Web 上の画像をワークシートに挿入するには、次のコードのようにローカルにダウンロードしたデータを `openpyxl.drawing.image.Image` のコンストラクタに渡す必要がある。スクリプトを実行すると、Colab のローカルファイルに `test_image.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [ ]:
import urllib.request
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from io import BytesIO
from PIL import Image as PilImage

# 画像をURLから取得
url = "https://upload.wikimedia.org/wikipedia/commons/0/0c/JR_east_485_hakutyo.jpg"

with urllib.request.urlopen(url) as response:
    img_data = response.read()
    # Pillowで画像オブジェクトとして読み込み
    pil_img = PilImage.open(BytesIO(img_data))

# 一時的に保存（openpyxl.drawing.image.Image はファイルパスが必要）
temp_path = "temp_image.jpg"
pil_img.save(temp_path)

# Excelへ挿入
wb = Workbook()
ws = wb.active
img = Image(temp_path)
ws.add_image(img, "A1")

# 保存
wb.save("test_image.xlsx")

from google.colab import files
files.download("/content/test_image.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

チャートの挿入
--------------

Excel のチャート（グラフ）と `openpyxl` のクラスの対応は、次のとおり。

| Excel | openpyxl |
|:---|:---|
| 棒グラフ | `openpyxl.chart.BarChart` |
| 棒(3D)グラフ | `openpyxl.chart.BarChart3D` |
| 折れ線グラフ | `openpyxl.chart.LineChart` |
| 折れ線(3D)グラフ | `openpyxl.chart.LineChart3D` |
| 円グラフ | `openpyxl.chart.PieChart` |
| ドーナツグラフ | `openpyxl.chart.DoughnutChart` |
| 面グラフ | `openpyxl.chart.AreaChart` |
| 面(3D)グラフ | `openpyxl.chart.AreaChart3D` |
| 散布図 | `openpyxl.chart.ScatterChart` |
| バブルチャート | `openpyxl.chart.BubbleChart` |
| 株価チャート | `openpyxl.chart.StockChart` |
| 等高線グラフ | `openpyxl.chart.SurfaceChart` |
| 等高線(3D)グラフ | `openpyxl.chart.SurfaceChart3D` |
| レーダーチャート | `openpyxl.chart.RadarChart` |

以下では、チャートオブジェクトを `chart` で表す。たとえば、折れ線グラフのオブジェクトは、次のように作成する。

``` python
from openpyxl.chart import LineChart
chart = LineChart()
```

チャートを作図するために参照するデータを**系列**または**シリーズ**（series）と呼ぶ。円グラフでは系列は 1 つだが、その他のチャートでは系列は複数あってもよい。

また、横軸の要素など、指標とするために参照するデータを**項目**または**カテゴリー**（categories）と呼ぶ。項目は、多くのグラフでは横軸の要素に使われるが、円グラフとドーナツグラフでは中心角の比率に、レーダーチャートでは項目に使われる。

系列および項目として参照するデータの範囲を指定するには、`openpyxl.chart.Reference` オブジェクトを使う。

``` python
openpyxl.chart.Reference(worksheet=None, min_col=None, min_row=None, max_col=None, max_row=None, range_string=None)
```

| 引数 | 意味 |
|:---|:---|
| `worksheet` | ワークシートを指定する |
| `min_col` | 参照範囲の最小の列番号（1 から始める） |
| `min_row` | 参照範囲の最小の行番号（1 から始める） |
| `max_col` | 参照範囲の最大の列番号 |
| `max_row` | 参照範囲の最大の行番号 |
| `range_string` | 参照範囲を、ワークシート名から始まる Excel の範囲指定文字列で指定する（例 `'Sheet!A2:A13'`）。この `range_string` に値を指定する場合、他の引数の値は使われない |

チャートオブジェクトに系列および項目を登録するには、以下のメソッドを呼び出す。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `chart.set_categories(labels)` | 項目を登録する。`labels` に `openpyxl.chart.Reference` インスタンスを指定する | `None` |
| `chart.add_data(data, from_rows=False,`<br />` titles_from_data=False)` | 系列を登録する。`data` に `openpyxl.chart.Reference` インスタンスを指定する。`from_rows` が `True` なら行を系列とするが、<br />`False`（デフォルト）なら列を系列とする。`titles_from_data` が `True` の場合、各系列の最初の要素をタイトルとして扱う | `None` |

``` python
from openpyxl.chart import Reference
data = Reference(ws, min_col=2, min_row=1, max_row=5)
categories = Reference(ws, min_col=1, min_row=2, max_row=5)
chart.add_data(data, titles_from_data=True)
chart.set_categories(categories)
```

すべてのチャートオブジェクトに共通する属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `chart.width` | グラフの幅。デフォルトは 15 cm |
| `chart.height` | グラフの高さ。デフォルトは 7.5 cm |
| `chart.title` | グラフのタイトル |
| `chart.legend` | グラフの凡例。自動的に作成される `Legend` インスタンスを参照し、次の属性にアクセスできるが、`legend` に `None` を指定すると凡例を表示しない<br />・`position`: `'b'`, `'l'`, `'tr'`, `'t'`, `'r'` を指定すると、それぞれグラフの下、左、右上、左上、右に凡例を配置する（デフォルトは `'r'`） |
| `chart.style` | グラフの配色を設定する。48 種類の組み込みスタイルの中から整数で選ぶ |
| `chart.series` | `chart.add_data()` メソッドの呼び出し時に作成される `Series` インスタンスのコレクションを参照する |

`Series` インスタンスは各系列のスタイルを管理するオブジェクトであり、次のように参照される:

``` python
s1 = chart.series[0]
```

円グラフおよびドーナツグラフ以外のグラフでは、軸に関する属性を持つ。

| 属性 | 意味 |
|:---|:---|
| `chart.x_axis` | グラフの横軸。`_BaseAxis` から継承するクラスのインスタンスを参照する |
| `chart.y_axis` | グラフの縦軸。`_BaseAxis` から継承するクラスのインスタンスを参照する |
| `chart.z_axis` | グラフの z 軸。等高線グラフと等高線(3D)グラフだけがこの属性を持つ。`_BaseAxis` から継承するクラスのインスタンスを参照する |

`_BaseAxis` クラスには、軸のタイトルを保持する `title` 属性が定義されている。

一通りの設定を終えたら、`Worksheet` オブジェクトの `add_chart(chart, anchor=None)` メソッドで、チャートを挿入する。

``` python
ws.add_chart(chart, "A15")
```

グラフの種類ごとに設定可能な属性については、[公式ドキュメント](https://openpyxl.readthedocs.io/en/stable/charts/introduction.html)を参照。

以下のコードでは、東京都の月ごとの平均気温のデータを参照して折れ線グラフを挿入する。スクリプトを実行すると、Colab のローカルファイルに `test_chart.xlsx` ファイルが作成され、自動的にダウンロードされる。

In [ ]:
from openpyxl import Workbook
from openpyxl.chart import LineChart, Reference

# 東京の月別平均気温
rows = [
    ["月", "2021年", "2022年"],
    ["1月", 5.4, 4.9],
    ["2月", 8.5, 5.2],
    ["3月", 12.8, 10.9],
    ["4月", 15.1, 15.3],
    ["5月", 19.6, 18.8],
    ["6月", 22.7, 23],
    ["7月", 25.9, 27.4],
    ["8月", 27.4, 27.5],
    ["9月", 22.3, 24.4],
    ["10月", 18.2, 17.2],
    ["11月", 13.7, 14.5],
    ["12月", 7.9, 7.5],
]

wb = Workbook()
ws = wb.active

for row in rows:
    ws.append(row)

chart = LineChart()
data = Reference(ws, min_col=2, min_row=1, max_col=3, max_row=13)
categories = Reference(range_string="Sheet!A2:A13")
chart.add_data(data, titles_from_data=True)
chart.set_categories(categories)

chart.title = "東京の月間平均気温"
chart.y_axis.title = "気温"
chart.x_axis.title = "月"

ws.add_chart(chart, "A15")

wb.save("test_chart.xlsx")

from google.colab import files
files.download("/content/test_chart.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ブックとシートの保護
--------------------

`openpyxl.Workbook` オブジェクトの作成時に、自動的に `DocumentSecurity` オブジェクトも作成されて、`openpyxl.Workbook` オブジェクトの `security` 属性でアクセスされる。そこで、以下のように `workbookPassword` 属性にパスワードを設定し、`lockStructure` 属性に `True` を設定すると、ワークブックが保護される（シートの挿入・削除・名前変更・並べ替え・複製ができなくなる）。

``` python
wb.security.workbookPassword = 'aaa'
wb.security.lockStructure = True
```

`Worksheet` オブジェクトの作成時に、自動的に `SheetProtection` のインスタンスも作成されて、`Worksheet` オブジェクトの `protection` 属性でアクセスされる。そこで、以下のように `sheet` 属性に `True` を設定し、`enable()` メソッドを呼び出すと、ワークシートが保護される。また、`disable()` メソッドを呼び出すと、保護が解除される。

``` python
ws.protection.sheet = True
ws.protection.enable()
ws.protection.disable()
```

ワークシート保護の解除にパスワードを要求する場合、以下のように `password` 属性にパスワードを設定する。

``` python
ws.protection.password = 'aaa'
```

ワークシートが保護されている場合に、`Cell` オブジェクトの `protection` 属性に `openpyxl.styles.Protection` オブジェクトを指定することで個別にセルの保護を無効化できる。

``` python
from openpyxl.styles import Protection
cell.protection = Protection(locked=False)
```

以下のコードは、`A1` のセルだけ編集可能で、他のセルは編集不可なシートを作成する:

``` python
from openpyxl import Workbook
from openpyxl.styles import Protection

wb = Workbook()
ws = wb.active

ws.cell(1, 1).value = 300
ws.cell(1, 1).protection = Protection(locked=False)  # ロックを解除
ws.cell(2, 1).value = 500

ws.protection.password = "aaa"
ws.protection.enable()

wb.save("test.xlsx")
```